In [56]:
from cffi import FFI

In [57]:
ffi = FFI()
ffi.cdef("""
    long __stdcall IV_open();
    long __stdcall IV_close();
    long __stdcall IV_MaxDevices();
    long __stdcall IV_selectdevice(long *devnr);
    long __stdcall IV_getdevicestatus();
    long __stdcall IV_readSN(char *sntext);
    long __stdcall IV_connect(long* devconnect);
    long __stdcall IV_VersionHost(long *version);
    long __stdcall IV_VersionDll();
    long __stdcall IV_VersionCheck();
    long __stdcall IV_HostHandle();
    long __stdcall IV_VersionDllFile();
    long __stdcall IV_VersionDllFileStr();
    long __stdcall IV_SelectChannel(long *channel);
""")

## Import dll

In [58]:
lib = ffi.dlopen("../pyvium_core/IVIUM_remdriver64.dll")

In [59]:
from dataclasses import dataclass

@dataclass
class App:
    is_driver_open: bool

app = App(False)

In [60]:
def iv_open():
    app.is_driver_open = True
    return lib.IV_open()

In [61]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class IviumSoftNotRunningError(Error):
    """Exception raised when the Ivium program is not open"""
    pass

class LibraryNotOpenError(Error):
    """Exception raised when the dll is not open"""
    pass
  
class DeviceNotConnectedError(Error):
    """Exception raised when no device is connected"""
    pass

def read_serial_number():
    serial_number = ffi.new("char[]", 16)
    rc = lib.IV_readSN(serial_number)

    print(app.is_driver_open)

    if not app.is_driver_open:
        raise LibraryNotOpenError

    if rc == -1:
        raise IviumSoftNotRunningError

    device_serial_number = ffi.string(serial_number).decode("utf-8")

    if device_serial_number == '':
        raise DeviceNotConnectedError

    return device_serial_number

In [63]:
iv_open()

try:
    print(read_serial_number())
except IviumSoftNotRunningError:
    print("Ivium is not running")
except LibraryNotOpenError:
    print("The library is closed")
except DeviceNotConnectedError:
    print("No device connected")

True
T53201
